In [5]:
# 소설 목록에서 스토리 크롤링 하기

In [6]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [7]:
# csv 파일 불러오기

In [8]:
list = pd.read_csv('novel_info_clean.csv')

In [9]:
list

,itemId,title,pubDate,coverSmallUrl,coverLargeUrl,publisher,priceStandard,customerReviewRank,author,isbn,link
0,281536876,파친코 1,20180309,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,문학사상,14500,9.5,이민진,9788970129815,http://book.interpark.com/blog/integration/pro...
1,281536885,파친코 2,20180309,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,문학사상,14500,9.2,이민진,9788970129822,http://book.interpark.com/blog/integration/pro...
2,348921505,불편한 편의점(40만부 기념 벚꽃 에디션),20210420,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,나무옆의자,14000,9.9,김호연,9791161571188,http://book.interpark.com/blog/integration/pro...
3,354595636,저주토끼,20220401,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,아작,14800,0.0,정보라(정도경),9791166686665,http://book.interpark.com/blog/integration/pro...
4,266107628,아몬드,20170331,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,창비,12000,9.6,손원평(孫元平),9788936434267,http://book.interpark.com/blog/integration/pro...
...,...,...,...,...,...,...,...,...,...,...,...
1113,255740918,군주론 3: 노무현 편(2003-2008),20160710,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,스토리뱅크,15000,0.0,이원호(李元浩),9788968402241,http://book.interpark.com/blog/integration/pro...
1114,230918620,사기꾼의 심장은 천천히 뛴다,20141201,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,알에이치코리아,11000,8.1,"곽재식(게렉터),곽재식",9788925554747,http://book.interpark.com/blog/integration/pro...
1115,353660972,머드(MUD),20210827,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,안전가옥,13000,0.0,이종산,9791191193176,http://book.interpark.com/blog/integration/pro...
1116,226451353,복자는 울지 않았다,20141110,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,삶창,12000,10.0,정낙추,9788966550449,http://book.interpark.com/blog/integration/pro...


In [10]:
# 아이디, 제목, 링크 칼럼만 뽑아오기

In [11]:
link_list = list[['itemId', 'title', 'link']]

In [12]:
link_list

,itemId,title,link
0,281536876,파친코 1,http://book.interpark.com/blog/integration/pro...
1,281536885,파친코 2,http://book.interpark.com/blog/integration/pro...
2,348921505,불편한 편의점(40만부 기념 벚꽃 에디션),http://book.interpark.com/blog/integration/pro...
3,354595636,저주토끼,http://book.interpark.com/blog/integration/pro...
4,266107628,아몬드,http://book.interpark.com/blog/integration/pro...
...,...,...,...
1113,255740918,군주론 3: 노무현 편(2003-2008),http://book.interpark.com/blog/integration/pro...
1114,230918620,사기꾼의 심장은 천천히 뛴다,http://book.interpark.com/blog/integration/pro...
1115,353660972,머드(MUD),http://book.interpark.com/blog/integration/pro...
1116,226451353,복자는 울지 않았다,http://book.interpark.com/blog/integration/pro...


In [13]:
# 링크 칼럼의 데이터 뽑기

In [14]:
link_value = link_list['link'].values

In [15]:
link_value

array(['http://book.interpark.com/blog/integration/product/itemDetail.rdo?prdNo=281536876&refererType=8303&bookblockname=bpmain_in&booklinkname=wg_search_55D3074F282255D523E67B9DA31B66C85AE3FEB55FD17DBEBE2AFAD4E770B9F1',
       'http://book.interpark.com/blog/integration/product/itemDetail.rdo?prdNo=281536885&refererType=8303&bookblockname=bpmain_in&booklinkname=wg_search_55D3074F282255D523E67B9DA31B66C85AE3FEB55FD17DBEBE2AFAD4E770B9F1',
       'http://book.interpark.com/blog/integration/product/itemDetail.rdo?prdNo=348921505&refererType=8303&bookblockname=bpmain_in&booklinkname=wg_search_55D3074F282255D523E67B9DA31B66C85AE3FEB55FD17DBEBE2AFAD4E770B9F1',
       ...,
       'http://book.interpark.com/blog/integration/product/itemDetail.rdo?prdNo=353660972&refererType=8303&bookblockname=bpmain_in&booklinkname=wg_search_55D3074F282255D523E67B9DA31B66C85AE3FEB55FD17DBEBE2AFAD4E770B9F1',
       'http://book.interpark.com/blog/integration/product/itemDetail.rdo?prdNo=226451353&refererType=83

In [16]:
links = link_value.tolist()

In [128]:
# 줄거리 1500개 전부 뽑아오기
from tqdm import tqdm

In [ ]:
all_book_data = []
need_ls = ["<p>책소개</p>", "<p>출판사 서평</p>", "<p>추천사</p>", "<p>목차</p>", "<p>본문중에서</p>"]

for i in tqdm(range(len(links))):
    response = requests.get(links[i])
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        all_find_p = soup.select('#bookInfoWrap > .detail_txtContent > .txtBox > p')
        find_h3 = soup.select('#inc_contentWrap > #bookInfoWrap > .detailTitle > p')
        all_find_h3 = []

        for h3 in find_h3:
            if str(h3) in need_ls:
                all_find_h3.append(h3)

        book_data = {}
        for p, h3 in zip(all_find_p,all_find_h3):
            p = str(p.get_text().strip())
            h3 = str(h3.get_text().strip())
            book_data[h3] = p

        all_book_data.append(book_data)

    else:
        print(response.status_code)

In [17]:
# 데이터 프레임으로 만들기

In [137]:
df = pd.DataFrame(all_book_data)

In [138]:
df

,책소개,출판사 서평,추천사,목차,본문중에서
0,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,구상부터 탈고까지 30년이 걸린 대작!차별받는 이민자의 투쟁적 삶을 일제강점기에서부...,주노 디아스(퓰리처상 수상작가)어둠 속에서도 빛나는 작품이다. 이민진의 소설 《파친...,"Book 1고향HOMETOWN부산의 작은 섬, 영도 ‥11한겨울의 방문자 ‥21젊은...",선자가 웃을 때면 같이 웃지 않고서는 배길 수가 없었다. 선자의 아버지 훈이는 선자...
1,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,구상부터 탈고까지 30년이 걸린 대작!차별받는 이민자의 투쟁적 삶을 일제강점기에서부...,주노 디아스(퓰리처상 수상작가)어둠 속에서도 빛나는 작품이다. 이민진의 소설 《파친...,"Book 2조국MOTHERLAND나쁜 조선인 … 9새로운 보스 … 23파친코 사장,...","노아는 엄청난 일을, 거의 불가능에 가까운 일을 해냈다. 그런데 그 모든 것을 무효..."
2,"※ 인터넷 한정 특별판: 매장 구매, 바로드림 구매 시에는 다른 표지의 에디션으로 ...",불편한데 자꾸 가고 싶은 편의점이 있다! 힘들게 살아낸 오늘을 위로하는 편의점의 밤...,정여울(『1일 1페이지 세상에서 가장 짧은 심리수업 365』 저자)서울역 홈리스로 ...,산해진미 도시락 제이에스 오브 제이에스 삼각김밥의 용도 원 플러스 원 불편한 편의점...,“근데 이게 마지막 술이에요. 이거 먹고 술 끊는 조건으로 우리 가게 일 좀 봐줘요...
3,2022 부커상 최종 후보 지명작!한국 호러 SF/판타지 대표작가 정보라 대표작20...,NaN,NaN,"저주토끼머리차가운 손가락몸하다안녕, 내 사랑덫흉터즐거운 나의 집바람과 모래의 지배자...",〈저주토끼〉할아버지는 늘 말씀하셨다. “저주에 쓰이는 물건일수록 예쁘게 만들어야 하...
4,괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...,감정을 느끼지 못하는 소년의 특별한 성장 \r『아몬드』의 주인공 ‘윤재’는 감정을...,NaN,프롤로그\r1부\r2부\r3부\r4부\r에필로그\r작가의 말,프롤로그 \r 나에겐 아몬드가 있다. \r당신에게도 있다. \r당신이 가장 소중하게...
...,...,...,...,...,...
1113,이완호의 소설『군주론』 제3권 《노무현 편(2003-2008)》. 1993년부터 2...,NaN,NaN,1장 13년 만의 외출\r2장 출구 없는 협상\r3장 최후통첩\r4장 피아 식별\r...,NaN
1114,"출장 중 우연히 알게 된 신기한 사실, 그리고 그 비밀로 인해 한 평범한 직장인의 ...","고전적 스타일과 현대적 플롯을 절묘하게 접목시킨, 당신이 단 한 번도 접해보지 못한...",NaN,01 어떻게 북회귀선을 통과한 태양의 고도가 기이한 이야기의 채록과 전파에 도움이 ...,하지만 지금 돌아보면 딱 거기까지였습니다. 대학을 다니면서 그 후로도 가끔 그녀와 ...
1115,“나는 이 사랑 때문에 죽을 거야.”바닥까지 가라앉았던 마음이 순식간에 목 끝까지 ...,"| 바닥까지 가라앉았던 마음이 순식간에 목 끝까지 튀어 오르고, 눈이 멀 정도로 눈...",NaN,1부완벽한 연결2부구속3부 도피4부 붕괴작가의 말프로듀서의 말,"등장이 요란하기도 하지, 내 사랑. 내가 세상에서 가장 사랑하고 또 미워하는 너. ..."
1116,정낙추 소설집『복자는 울지 않았다』. 개발과 이윤의 논리에 훼손당한 이 시대의 자화...,‘개발’과 ‘이윤’의 논리로 가득한 세상 속에서\r생동하는 민중의 풀빛 카니발 ...,NaN,복자는 울지 않았다 _ 7\r오빠 생각 _ 83\r끈 _ 113\r죄인 _ 191\...,농사도 짓지 않는 사람들이 땅을 얼마나 사랑하기에 입길에 오르며 땅을 사는지 모르지...


In [139]:
# 기존 데이터 프레임과 합치기

In [140]:
result = pd.concat([link_list, df], axis=1)

In [141]:
result

,itemId,title,link,책소개,출판사 서평,추천사,목차,본문중에서
0,281536876,파친코 1,http://book.interpark.com/blog/integration/pro...,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,구상부터 탈고까지 30년이 걸린 대작!차별받는 이민자의 투쟁적 삶을 일제강점기에서부...,주노 디아스(퓰리처상 수상작가)어둠 속에서도 빛나는 작품이다. 이민진의 소설 《파친...,"Book 1고향HOMETOWN부산의 작은 섬, 영도 ‥11한겨울의 방문자 ‥21젊은...",선자가 웃을 때면 같이 웃지 않고서는 배길 수가 없었다. 선자의 아버지 훈이는 선자...
1,281536885,파친코 2,http://book.interpark.com/blog/integration/pro...,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,구상부터 탈고까지 30년이 걸린 대작!차별받는 이민자의 투쟁적 삶을 일제강점기에서부...,주노 디아스(퓰리처상 수상작가)어둠 속에서도 빛나는 작품이다. 이민진의 소설 《파친...,"Book 2조국MOTHERLAND나쁜 조선인 … 9새로운 보스 … 23파친코 사장,...","노아는 엄청난 일을, 거의 불가능에 가까운 일을 해냈다. 그런데 그 모든 것을 무효..."
2,348921505,불편한 편의점(40만부 기념 벚꽃 에디션),http://book.interpark.com/blog/integration/pro...,"※ 인터넷 한정 특별판: 매장 구매, 바로드림 구매 시에는 다른 표지의 에디션으로 ...",불편한데 자꾸 가고 싶은 편의점이 있다! 힘들게 살아낸 오늘을 위로하는 편의점의 밤...,정여울(『1일 1페이지 세상에서 가장 짧은 심리수업 365』 저자)서울역 홈리스로 ...,산해진미 도시락 제이에스 오브 제이에스 삼각김밥의 용도 원 플러스 원 불편한 편의점...,“근데 이게 마지막 술이에요. 이거 먹고 술 끊는 조건으로 우리 가게 일 좀 봐줘요...
3,354595636,저주토끼,http://book.interpark.com/blog/integration/pro...,2022 부커상 최종 후보 지명작!한국 호러 SF/판타지 대표작가 정보라 대표작20...,NaN,NaN,"저주토끼머리차가운 손가락몸하다안녕, 내 사랑덫흉터즐거운 나의 집바람과 모래의 지배자...",〈저주토끼〉할아버지는 늘 말씀하셨다. “저주에 쓰이는 물건일수록 예쁘게 만들어야 하...
4,266107628,아몬드,http://book.interpark.com/blog/integration/pro...,괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...,감정을 느끼지 못하는 소년의 특별한 성장 \r『아몬드』의 주인공 ‘윤재’는 감정을...,NaN,프롤로그\r1부\r2부\r3부\r4부\r에필로그\r작가의 말,프롤로그 \r 나에겐 아몬드가 있다. \r당신에게도 있다. \r당신이 가장 소중하게...
...,...,...,...,...,...,...,...,...
1113,255740918,군주론 3: 노무현 편(2003-2008),http://book.interpark.com/blog/integration/pro...,이완호의 소설『군주론』 제3권 《노무현 편(2003-2008)》. 1993년부터 2...,NaN,NaN,1장 13년 만의 외출\r2장 출구 없는 협상\r3장 최후통첩\r4장 피아 식별\r...,NaN
1114,230918620,사기꾼의 심장은 천천히 뛴다,http://book.interpark.com/blog/integration/pro...,"출장 중 우연히 알게 된 신기한 사실, 그리고 그 비밀로 인해 한 평범한 직장인의 ...","고전적 스타일과 현대적 플롯을 절묘하게 접목시킨, 당신이 단 한 번도 접해보지 못한...",NaN,01 어떻게 북회귀선을 통과한 태양의 고도가 기이한 이야기의 채록과 전파에 도움이 ...,하지만 지금 돌아보면 딱 거기까지였습니다. 대학을 다니면서 그 후로도 가끔 그녀와 ...
1115,353660972,머드(MUD),http://book.interpark.com/blog/integration/pro...,“나는 이 사랑 때문에 죽을 거야.”바닥까지 가라앉았던 마음이 순식간에 목 끝까지 ...,"| 바닥까지 가라앉았던 마음이 순식간에 목 끝까지 튀어 오르고, 눈이 멀 정도로 눈...",NaN,1부완벽한 연결2부구속3부 도피4부 붕괴작가의 말프로듀서의 말,"등장이 요란하기도 하지, 내 사랑. 내가 세상에서 가장 사랑하고 또 미워하는 너. ..."
1116,226451353,복자는 울지 않았다,http://book.interpark.com/blog/integration/pro...,정낙추 소설집『복자는 울지 않았다』. 개발과 이윤의 논리에 훼손당한 이 시대의 자화...,‘개발’과 ‘이윤’의 논리로 가득한 세상 속에서\r생동하는 민중의 풀빛 카니발 ...,NaN,복자는 울지 않았다 _ 7\r오빠 생각 _ 83\r끈 _ 113\r죄인 _ 191\...,농사도 짓지 않는 사람들이 땅을 얼마나 사랑하기에 입길에 오르며 땅을 사는지 모르지...


In [44]:
# link 칼럼만 없애기

In [142]:
just = result.drop(columns=["link"])

In [143]:
just

,itemId,title,책소개,출판사 서평,추천사,목차,본문중에서
0,281536876,파친코 1,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,구상부터 탈고까지 30년이 걸린 대작!차별받는 이민자의 투쟁적 삶을 일제강점기에서부...,주노 디아스(퓰리처상 수상작가)어둠 속에서도 빛나는 작품이다. 이민진의 소설 《파친...,"Book 1고향HOMETOWN부산의 작은 섬, 영도 ‥11한겨울의 방문자 ‥21젊은...",선자가 웃을 때면 같이 웃지 않고서는 배길 수가 없었다. 선자의 아버지 훈이는 선자...
1,281536885,파친코 2,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,구상부터 탈고까지 30년이 걸린 대작!차별받는 이민자의 투쟁적 삶을 일제강점기에서부...,주노 디아스(퓰리처상 수상작가)어둠 속에서도 빛나는 작품이다. 이민진의 소설 《파친...,"Book 2조국MOTHERLAND나쁜 조선인 … 9새로운 보스 … 23파친코 사장,...","노아는 엄청난 일을, 거의 불가능에 가까운 일을 해냈다. 그런데 그 모든 것을 무효..."
2,348921505,불편한 편의점(40만부 기념 벚꽃 에디션),"※ 인터넷 한정 특별판: 매장 구매, 바로드림 구매 시에는 다른 표지의 에디션으로 ...",불편한데 자꾸 가고 싶은 편의점이 있다! 힘들게 살아낸 오늘을 위로하는 편의점의 밤...,정여울(『1일 1페이지 세상에서 가장 짧은 심리수업 365』 저자)서울역 홈리스로 ...,산해진미 도시락 제이에스 오브 제이에스 삼각김밥의 용도 원 플러스 원 불편한 편의점...,“근데 이게 마지막 술이에요. 이거 먹고 술 끊는 조건으로 우리 가게 일 좀 봐줘요...
3,354595636,저주토끼,2022 부커상 최종 후보 지명작!한국 호러 SF/판타지 대표작가 정보라 대표작20...,NaN,NaN,"저주토끼머리차가운 손가락몸하다안녕, 내 사랑덫흉터즐거운 나의 집바람과 모래의 지배자...",〈저주토끼〉할아버지는 늘 말씀하셨다. “저주에 쓰이는 물건일수록 예쁘게 만들어야 하...
4,266107628,아몬드,괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...,감정을 느끼지 못하는 소년의 특별한 성장 \r『아몬드』의 주인공 ‘윤재’는 감정을...,NaN,프롤로그\r1부\r2부\r3부\r4부\r에필로그\r작가의 말,프롤로그 \r 나에겐 아몬드가 있다. \r당신에게도 있다. \r당신이 가장 소중하게...
...,...,...,...,...,...,...,...
1113,255740918,군주론 3: 노무현 편(2003-2008),이완호의 소설『군주론』 제3권 《노무현 편(2003-2008)》. 1993년부터 2...,NaN,NaN,1장 13년 만의 외출\r2장 출구 없는 협상\r3장 최후통첩\r4장 피아 식별\r...,NaN
1114,230918620,사기꾼의 심장은 천천히 뛴다,"출장 중 우연히 알게 된 신기한 사실, 그리고 그 비밀로 인해 한 평범한 직장인의 ...","고전적 스타일과 현대적 플롯을 절묘하게 접목시킨, 당신이 단 한 번도 접해보지 못한...",NaN,01 어떻게 북회귀선을 통과한 태양의 고도가 기이한 이야기의 채록과 전파에 도움이 ...,하지만 지금 돌아보면 딱 거기까지였습니다. 대학을 다니면서 그 후로도 가끔 그녀와 ...
1115,353660972,머드(MUD),“나는 이 사랑 때문에 죽을 거야.”바닥까지 가라앉았던 마음이 순식간에 목 끝까지 ...,"| 바닥까지 가라앉았던 마음이 순식간에 목 끝까지 튀어 오르고, 눈이 멀 정도로 눈...",NaN,1부완벽한 연결2부구속3부 도피4부 붕괴작가의 말프로듀서의 말,"등장이 요란하기도 하지, 내 사랑. 내가 세상에서 가장 사랑하고 또 미워하는 너. ..."
1116,226451353,복자는 울지 않았다,정낙추 소설집『복자는 울지 않았다』. 개발과 이윤의 논리에 훼손당한 이 시대의 자화...,‘개발’과 ‘이윤’의 논리로 가득한 세상 속에서\r생동하는 민중의 풀빛 카니발 ...,NaN,복자는 울지 않았다 _ 7\r오빠 생각 _ 83\r끈 _ 113\r죄인 _ 191\...,농사도 짓지 않는 사람들이 땅을 얼마나 사랑하기에 입길에 오르며 땅을 사는지 모르지...


In [47]:
# csv 파일로 만들기

In [48]:
just.to_csv("novel_story.csv", header = True, index = False, encoding='utf-8-sig')